# CLUSTER para agrupar las paradas de los hornos en conjuntos en función de su duración.

## El objetivo principal es calcular las fronteras temporales entre cada tipo de parada. En posteriores versiones se buscará definir el tipo dominante en cada cluster, probablemente con otra estrategia diferente a K-MEANS

## Se escoge K-MEANS porque previamente se decide el número de clusters tras un estudio visual.

## Se crean grupos en función de la duración de la parada. En el modelo de la parada del horno, las paradas, en función de su duración tendrán las características del tipo dominante en cada agrupamiento por duración

## Se calcula la frontera temporal entre tipos de parada en función de los centros de cada cluster y de sus bordes. 

In [1]:
# external packages
import datetime as dt
import pandas as pd
import numpy as np
import math
import operator

%matplotlib widget
import matplotlib.pyplot as plt
np.seterr(divide='ignore', invalid='ignore')

from matplotlib.dates import drange
import datetime 
import itertools

In [30]:
from sklearn.cluster import KMeans 
import sklearn.utils
from sklearn.preprocessing import StandardScaler

# Cluster K-MEANS

### Se crea el cluster como un elemento previo de preprocesamiento del algoritmo K-MEANS

In [1]:
def crea_cluster_previo(parada_n_HH):
    print("\n")
    print("CLUSTER CREADO en función de la duración de la parada")
    duracion_ = [p.duracion_parada for p in parada_n_HH]
    consumo_ = [p.caida_media for p in parada_n_HH]
#     plt.figure()
#     plt.scatter(duracion_, consumo_  ,c='DarkBlue')
#     plt.title('Correlación entre duración parada horno  y caida del consumo')
#     plt.ylabel('KW bajada')
#     plt.xlabel('Horas')

    cluster_paradas_HH = pd.DataFrame({'duracion' : duracion_ , 'consumo' : consumo_, 'Clus_Db':np.zeros(len(consumo_)) })
    cluster_paradas_HH.to_excel('cluster_paradas_HH.xlsx', engine='xlsxwriter')  
    filename='cluster_paradas_HH.xlsx'
    #Read csv
    cluster_paradas_HH = pd.read_excel(filename)
    cluster_paradas_HH  = cluster_paradas_HH .drop(cluster_paradas_HH .columns[[0]], axis=1)
    #print(cluster_paradas_HH.head(5))
    return cluster_paradas_HH

### Ahora se crean los cluster según el algoritmo K-MEANS, se obitneen los valores máximos y mínimos de duración para calcular posteriormente las fronteras entre clusters

In [2]:
def cluster_KMEANS(cluster_paradas_HH, clusterNum):
    X = cluster_paradas_HH.values[:,:-2] # Seleccionar SOLO por duracion de parada
    #X = cluster_paradas_HH.values[:,:-1] # Seleccionar por duracion y por consumo
    X = np.nan_to_num(X)
    
    k_means = KMeans(init = "k-means++", n_clusters = clusterNum, n_init = 10)
    k_means.fit(X)
    labels = k_means.labels_
    cluster_paradas_HH["Clus_Db"] = labels #Asignacion de la categoria a cada elemento del Cluster

    unique_labels = set(labels)
    print(clusterNum)
    print(labels)
    print(unique_labels)

    colors = plt.cm.Spectral(np.linspace(0, 1, len(unique_labels)))
    
    #Visualization
    plt.figure()
    plt.title('K-Means Cluster Horno')
    plt.ylabel('KW bajada')
    plt.xlabel('Horas')

    min_cluster = []
    max_cluster = []   
    for clust_number in set(labels):
        c=(([0.4,0.4,0.4]) if clust_number == -1 else colors[int(clust_number)])
        clust_set = cluster_paradas_HH[cluster_paradas_HH.Clus_Db == clust_number]                    
        plt.scatter(clust_set.duracion, clust_set.consumo, color =c,  marker='o', s= 20, alpha = 0.85)
        cenx=np.mean(clust_set.duracion) 
        ceny=np.mean(clust_set.consumo) 
        print ("Cluster "+str(clust_number)+', Avg Druacion: '+ str(np.mean(clust_set.duracion))+', Avg Consumo: '+ str(np.mean(clust_set.consumo)))
        min_cluster.append(np.min(clust_set.duracion))
        max_cluster.append(np.max(clust_set.duracion))
        
    return k_means,min_cluster,max_cluster    

### Se calculan las fronteras temporales según K-MEANS

In [1]:
def calcula_frontera_horaria_tipos(k_means,clust_number,min_cluster,max_cluster):
    
    Frontera_centro = []
    for i in range(0,clust_number-1):
        Frontera_centro.append(np.rint(((k_means.cluster_centers_[i] + k_means.cluster_centers_[i+1])/2)[0]))
    print("Frontera centro",Frontera_centro) #Referencia en los centros del cluster
    
    Frontera_bordes = []
    for i in range(0,clust_number-1):
        Frontera_bordes.append(np.rint((max_cluster[i] + min_cluster[i+1])/2))
    print("Frontera bordes",Frontera_bordes) #Referencia en los bordes del cluster
    
    return(Frontera_centro,Frontera_bordes)